In [ ]:
# The pupose of this notebook is to test the updation of the emebeddings

In [1]:
import pandas as pd
import numpy as np
import re
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models.doc2vec import Doc2Vec
import random
import string
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from scipy import spatial
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
lemmatizer = WordNetLemmatizer()
import math

In [3]:
vec_one = np.ones(768)

In [4]:
vec_zero = np.zeros(768)

In [7]:
######## For post Doc method either 768 embedding of tweets need to go, or meta deta will have to be zero ###########

In [5]:
vec_tweetupdate = np.concatenate((vec_zero,vec_one), axis = None) 
MatrixForTweetUpdate = np.diag(vec_tweetupdate)

In [104]:
Main_df = pd.read_csv('C:/Users/SHAHBAZ/ZS UserEmbedding/BERT Embedding/MainDF.csv')
Main_df = Main_df[["AUTHOR_ID","TRANS_AUTHOR_BIO","Snippet"]]

In [107]:
Main_df.to_csv('test.csv', header=True, index=False) 

In [105]:
print("Author Name  :" + str(rowd['AUTHOR_ID']))
print("Author Info  :" + str(rowd['TRANS_AUTHOR_BIO']))
print("Tweet  :" + str(rowd['Snippet']))

Author Name  :433    CDFrazier87
Name: AUTHOR_ID, dtype: object
Author Info  :433    ['B.A., @DickinsonCol and M.A./Ph.D., @Georget...
Name: TRANS_AUTHOR_BIO, dtype: object
Tweet  :433    ['Diabetes, asthma, and heart disease, can res...
Name: Snippet, dtype: object


In [7]:
def getlabelforUser(userEmb,categoryList):
    listScoreUser = []
    for category in categoryList:
        listScoreUser.append(measureSimilarityMeanofUser(userEmb,category))    
    m =  max(listScoreUser)    
    return (listScoreUser.index(m) + 1)

from scipy import spatial
def measureSimilarityMeanofUser(userEmb,category):
    sumsc = 0
    for user in category:
        res = 1 - spatial.distance.cosine(user,userEmb)
        sumsc = sumsc + res
    
    mean = sumsc / len(category)
    return mean

import random
def customSplit(category,num):
    random.Random(7).shuffle(category)
    return category[:num]

In [8]:
def ReadMatfromPath(fileName):
    ReadFilepath = "C:/Users/SHAHBAZ/ZS UserEmbedding/BERT Embedding/" + str(fileName) + "_Matrix.txt"
    UserEmb = np.loadtxt(ReadFilepath)
    return UserEmb

In [9]:
############### Building the categories for mapping #################

In [10]:
train_df = pd.read_csv('TrainData_1.csv')
train_df['Embedding'] = ""
train_df = train_df.dropna()

In [11]:
def PreprareTrainDataEmb(Mat,authorId):
    rowd = Main_df.loc[Main_df['AUTHOR_ID'] == authorId]
    ind = rowd.index.values.astype(int)[0]
    return Mat[ind]

In [12]:
def CombineMatrix(Mat1,Mat2):
    Mat = np.zeros((Mat1.shape[1] + Mat2.shape[1]))
    for i in range (1,Mat1.shape[0]):
        vec = []
        vec = np.concatenate((vec,Mat1[i]), axis=None)
        vec = np.concatenate((vec,Mat2[i]), axis=None)
        Mat = np.vstack((Mat,vec))
    return Mat    

In [18]:
train_df['Embedding'] = [PreprareTrainDataEmb(BioBert1636,x['AUTHOR_ID']) for _,x in train_df.iterrows()]

In [19]:
    category1 = []   #28     Nurse
    category2 = []   #119    doctors
    category3 = []   #80     prof student organisation
    category4 = []   #72     others

    for _,trow in train_df.iterrows():
        if(trow['Label'] == 1.0):
            category1.append(trow['Embedding'])
        if(trow['Label'] == 2.0):
            category3.append(trow['Embedding'])
        if(trow['Label'] == 3.0):
            category2.append(trow['Embedding'])
        if(trow['Label'] == 4.0):
            category2.append(trow['Embedding'])
        if(trow['Label'] == 5.0):
            category4.append(trow['Embedding'])
        if(trow['Label'] == 6.0):
            category2.append(trow['Embedding'])
        if(trow['Label'] == 7.0):
            category3.append(trow['Embedding'])
        if(trow['Label'] == 8.0):
            category3.append(trow['Embedding'])

In [20]:
category1 = customSplit(category1,25)
category2 = customSplit(category2,25)
category3 = customSplit(category3,25)
category4 = customSplit(category4,25)

In [21]:
categoryList = []
categoryList.append(category1)
categoryList.append(category2)
categoryList.append(category3)
categoryList.append(category4)

In [31]:
############## Testing for bioBert 1536 + 100 dim deepwalk embedding ##########

In [13]:
DeepWalkEmb = ReadMatfromPath('DeepWalk')   
BioBertSnippetEmbMat = ReadMatfromPath('BioBERT_SnippetEmb')
BioBertAuthorInfoEmbMat = ReadMatfromPath('BioBERT_Autho_Info_Embedding')

In [15]:
BioBertSnippetEmbMat = BioBertSnippetEmbMat[1:,:]
BioBertAuthorInfoEmbMat = BioBertAuthorInfoEmbMat[1:,:]
DeepWalkEmb = DeepWalkEmb[:4280,:]

In [16]:
BioBert1536 = CombineMatrix(BioBertSnippetEmbMat,BioBertAuthorInfoEmbMat)

In [17]:
BioBert1636 = CombineMatrix(BioBert1536,DeepWalkEmb)

In [25]:
rowd = Main_df.loc[Main_df['AUTHOR_ID'] == 'CDFrazier87']
ind = rowd.index.values.astype(int)[0]
ind

433

In [26]:
original_emb = BioBert1636[ind]

In [27]:
getlabelforUser(original_emb,categoryList)   #correctly classified to Student

3

In [28]:
rowd = Main_df.loc[Main_df['AUTHOR_ID'] == 'BrukeGetachew']
ind = rowd.index.values.astype(int)[0]
ind

420

In [29]:
original2_emb = BioBert1636[ind]

In [30]:
getlabelforUser(original2_emb,categoryList) # correct classified as others is a data scientist 

4

In [37]:
rowd = Main_df.loc[Main_df['AUTHOR_ID'] == 'Beth17459893']
ind = rowd.index.values.astype(int)[0]
ind

336

In [38]:
target_emb = BioBert1636[ind]

In [39]:
getlabelforUser(target_emb,categoryList) # correct classified as Nurse 

1

In [64]:
rowd = Main_df.loc[Main_df['AUTHOR_ID'] == 'AuntLowlen']
ind = rowd.index.values.astype(int)[0]
ind

271

In [65]:
target2_emb = BioBert1636[ind]

In [66]:
getlabelforUser(target2_emb,categoryList)  # classified since professor

3

In [69]:
DeepWalkEmb[433]

array([-0.01049469, -0.15814716, -0.49185997, -0.09459099, -0.4635784 ,
        0.18302421, -0.64892507, -0.42392343, -0.25053915, -0.39454412,
       -0.60996133,  0.08691426,  0.08658071,  0.38523731,  0.09279715,
       -0.75825781, -0.13066663, -0.1792139 , -0.42092815, -0.0130418 ,
       -0.18073395, -0.46512792,  0.0489935 ,  0.09113819,  0.2206409 ,
        0.12321874, -0.48454323, -0.24975935, -0.21950181,  0.8278265 ,
        0.37762299,  0.02836859,  0.37850013, -0.26743799,  0.59439057,
        0.214417  , -0.19457734,  0.23971429,  0.22797388,  0.38492185,
        0.14186861, -0.6096555 ,  0.15769678,  0.04523146,  1.03874218,
       -0.42761239,  0.25639629, -0.78074664,  0.62243187,  0.14092943,
        0.17973492, -0.21092811, -0.35285714, -0.40570223,  0.03142745,
        0.07292751, -0.41196436,  0.36316538, -0.35537586,  0.60011172,
       -0.1879559 ,  0.03515817, -0.10196326, -0.71267188, -0.25009611,
        0.24955718, -0.72345257, -0.13628791,  0.18498449, -0.48

In [70]:
TestMat = np.zeros(1536)

In [76]:
vec = []
vec = np.concatenate((BioBertSnippetEmbMat[420],BioBertSnippetEmbMat[270]))
TestMat = np.vstack((TestMat,vec))

In [78]:
TestMat = TestMat[1:,:]

In [79]:
Savefilepath = "C:/Users/SHAHBAZ/ZS UserEmbedding/AutoEncoderInputUserMatrix/"
fileName = Savefilepath + "New_Update_Testing_1536.txt"
np.savetxt(fileName,TestMat,fmt='%.8f') 

In [80]:
#######load Mat#########

In [81]:
DeepWalkUpdated_Mat = ReadMatfromPath('DeepWalkForUpdate')   
UpdatedSnippetEmb_BioBERT = ReadMatfromPath('Update_Testing_768') 

In [94]:
new_original_updated1 = []
new_original_updated1 = np.concatenate((new_original_updated1,UpdatedSnippetEmb_BioBERT[3]),axis=None)
new_original_updated1 = np.concatenate((new_original_updated1,BioBertAuthorInfoEmbMat[420]),axis=None)
new_original_updated1 = np.concatenate((new_original_updated1,DeepWalkUpdated_Mat[3]),axis=None)
new_original_updated1.shape

(1636,)

In [97]:
getlabelforUser(new_original_updated1,categoryList)

3